Models were automatically reconstructed from NCBI RefSeq accession codes with carveme.

For NJ4:

```
carve --refseq GCF_003014985.1 --gapfill nj4_med -i nj4_med --mediadb medium.tsv -o GEMs/NJ4.xml 
```

W/o gapfill - no growth was possible on the xylose medium.

In [54]:
from cobra.io import read_sbml_model
from utils import model_validation as mv

nj4 = read_sbml_model('GEMs/NJ4.xml')

In [13]:
test_model = nj4.copy()

In [16]:
sol = nj4.slim_optimize()

In [18]:
type(sol)

float

writing a function for A->B testing using sink / demands.

In [19]:
def validate_pathway(model, A, B) -> bool:
    """Tests whether the model network supports production of metabolite B from metabolite A."""
    
    test_model = model.copy()

    test_model.add_boundary(test_model.metabolites.get_by_id(A), type="sink", reaction_id="SK_A")
    test_model.add_boundary(test_model.metabolites.get_by_id(B), type="sink", reaction_id="DM_B")

    test_model.objective="DM_B"

    sol = test_model.slim_optimize()
    return sol > 0 

In [22]:
validate_pathway(nj4, "glc__D_c", "pyr_c")

True

# Testing the model:

1. check whether growth is possible on the medium,  
use ```growth_possible```.

1. check whether the exhange reactions exist in the model,  
use ```reactions_exist```.

2. check whether the exhange reactions are blocked,  
use ```check_blocked_reactions```.

2. check whether the exhange reactions can carry flux at optimal solution (FVA),  
use ```check_production```.

In [39]:
mv.growth_possible(nj4)

True

In [40]:
uptake_reactions = [
    'EX_xyl__D_e', # xylose exchange
]

production_reactions = [
    'EX_btoh_e', # butanol exhange
    'EX_etoh_e', # ethanol exchange
    'EX_ac_e', # acetate exchange
    'EX_but_e', # butyrate exchange
    'EX_acetone_e', # acetone exchange
]

In [42]:
mv.reactions_exist(nj4, uptake_reactions)

True

In [43]:
mv.reactions_exist(nj4, production_reactions)

The following reactions are missing: ['EX_btoh_e', 'EX_acetone_e']


False

In [44]:
existing_production_reactions = [
    'EX_etoh_e', # ethanol exchange
    'EX_ac_e', # acetate exchange
    'EX_but_e', # butyrate exchange
]

In [45]:
mv.check_blocked_reactions(nj4, existing_production_reactions)

['EX_ac_e', 'EX_but_e']

In [46]:
mv.check_production(nj4, existing_production_reactions)

,minimum,maximum
EX_etoh_e,15.445155,15.844752
EX_ac_e,0.000000,0.000000
EX_but_e,0.000000,0.000000


The out-of-the-box carveme model:
1. can grow on the medium
1. does not contain the exhange reactions for butanol or acetone
1. does not contain acetone or butyrate production in its solution space
1. does produce ethanol

## Troubleshooting the carve-me generated model 
using ABE pathway overview w/ BiGG identifiers from notespace

In [54]:
metabolites = ['btoh', '1btol', 'btal', 'btcoa', 'butpi', 'but', 'acetone', 'acac', 'aacoa', 'b2coa', '3hbcoa', 'accoa', 'pyr']

for metabolite in metabolites:
    try:
        nj4.metabolites.get_by_id(str(metabolite+"_c"))
    except KeyError:
        print(metabolite, "not in model")

acetone not in model


all the metabolites required for the correct behaviour is present in the model - appart from acetone

In [72]:
reactions = ['HBCO_nadp','ACOAD1f', 'BTCOARx', 'ALCD4', 'BTS', 'PBUTT', 'BUTKr', 'BUTCT', 'BUTCT','EX_acetone_e', 'EX_but_e', 'EX_btoh_e']

for reaction in reactions:
    try:
        nj4.reactions.get_by_id(reaction)
    except KeyError:
        print(reaction, "not in model")

EX_acetone_e not in model


In [120]:
r1 = universal_model.metabolites.get_by_id("acac_c").reactions
r2 = universal_model.metabolites.get_by_id("acetone_c").reactions

# get list of elements in both r1 and r2
common = list(set(r1).intersection(r2))
print([r.id for r in common])

['ADCi']


In [137]:
nj4.reactions.get_by_id("BUTCT2")

Reaction identifier,BUTCT2
Name,
Memory address,0x14e603fd0
Stoichiometry,acac_c + btcoa_c <=> aacoa_c + but_c Acetoacetate + Butanoyl-CoA <=> Acetoacetyl-CoA + Butyrate (n-C4:0)
GPR,WP_010890847_1 and WP_010890848_1
Lower bound,-1000
Upper bound,1000


In [136]:
nj4.reactions.get_by_id("BUTCT")

Reaction identifier,BUTCT
Name,Acetyl-CoA:butyrate-CoA transferase
Memory address,0x14e603f70
Stoichiometry,accoa_c + but_c --> ac_c + btcoa_c Acetyl-CoA + Butyrate (n-C4:0) --> Acetate + Butanoyl-CoA
GPR,WP_010890847_1 or (WP_010890847_1 and WP_010890848_1)
Lower bound,0.0
Upper bound,1000.0


In [109]:
nj4.metabolites.get_by_id("aacoa_c")

Metabolite identifier,aacoa_c
Name,Acetoacetyl-CoA
Memory address,0x12e948370
Formula,C25H36N7O18P3S
Compartment,C_c
In 7 reaction(s),"HACD1i, ACACT1r, ACACCT, HBCO_nadp, OCOAT1, BUTCT2, HACD1"


In [113]:
nj4.reactions.get_by_id("ACACCT")

Reaction identifier,ACACCT
Name,Acetyl-CoA:acetoacetyl-CoA transferase
Memory address,0x12ed37ee0
Stoichiometry,acac_c + accoa_c --> aacoa_c + ac_c Acetoacetate + Acetyl-CoA --> Acetoacetyl-CoA + Acetate
GPR,WP_010890847_1 or (WP_010890847_1 and WP_010890848_1)
Lower bound,0.0
Upper bound,1000.0


In [75]:
but_model = nj4.copy()
but_model.objective = "EX_btoh_e"
but_model.optimize().objective_value

0.0

- maybe butanol production will be possible if adding transport and exhange reactions for butanol? try this then re-run the validation check
- there are two butanol forms - use the btoh / ALCD4 combo for this (it should really not matter since both are converted from the same butanal)

updating the model (now tried to fix up the butanol and acetone pathways)

In [55]:
from config import ROOT_DIR

universal_model = read_sbml_model(str(ROOT_DIR / "community_modelling" / "GEMs" / "bigg_universe.xml"))

# butabol transport reaction
BTOHt = universal_model.reactions.get_by_id('BTOHt')

# butyrate transport reaction
BUTt = universal_model.reactions.get_by_id('BUTt')

# acetoacetate -> acetone + CO2
ADCi = universal_model.reactions.get_by_id("ADCi")

# acetone transport reaction #TODO: figure out if this should be a proton symport
ACEt = universal_model.reactions.get_by_id("ACEt")


# acetoacetyl-CoA + butyrate -> acetoacetate + butanoyl-CoA
# BUTCT2 = universal_model.reactions.get_by_id('BUTCT2')
nj4.reactions.get_by_id("BUTCT2").bounds = (-1000, 1000) # make the reaction reversible


nj4.add_reactions([BTOHt, BUTt, ADCi, ACEt])
nj4.add_boundary(nj4.metabolites.get_by_id('btoh_e'), type='exchange', reaction_id='EX_btoh_e');
nj4.add_boundary(nj4.metabolites.get_by_id('acetone_e'), type='exchange', reaction_id='EX_acetone_e');

No objective coefficients in model. Unclear what should be optimized
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


In [24]:
from cobra.io import write_sbml_model
write_sbml_model(nj4, "GEMs/NJ4_curated.xml")

In [56]:
existing_production_reactions = [
    'EX_etoh_e', # ethanol exchange
    'EX_acetone_e', # acetone exchange
    'EX_ac_e', # acetate exchange
    'EX_but_e', # butyrate exchange
    'EX_btoh_e', # butanol exhange
]

mv.check_blocked_reactions(nj4, existing_production_reactions)

['EX_ac_e', 'EX_but_e']

In [52]:
universal_model.reactions.BTOHt

Reaction identifier,BTOHt
Name,Butanol transport by diffusion
Memory address,0x13493fe20
Stoichiometry,btoh_e <=> btoh_c Butanol <=> Butanol
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [59]:
from cobra.flux_analysis import gapfill

with nj4:
    nj4.objective = nj4.reactions.EX_but_e
    solution = gapfill(nj4, universal_model)
    for reaction in solution[0]:
        print(reaction.id)

BUTt


In [63]:
nj4.reactions.BUTtex

Reaction identifier,BUTtex
Name,Butyrate transport via diffusion (extracellular to periplasm)
Memory address,0x159a7a350
Stoichiometry,but_e <=> but_p Butyrate (n-C4:0) <=> Butyrate (n-C4:0)
GPR,WP_010963628_1
Lower bound,-1000.0
Upper bound,1000.0


In [62]:
nj4.metabolites.but_e

Metabolite identifier,but_e
Name,Butyrate (n-C4:0)
Memory address,0x15988f730
Formula,C4H7O2
Compartment,C_e
In 2 reaction(s),"BUTtex, EX_but_e"


In [57]:
import cobra

fva_sol = cobra.flux_analysis.flux_variability_analysis(nj4, existing_production_reactions, fraction_of_optimum=0)

In [58]:
fva_sol

,minimum,maximum
EX_etoh_e,0.000000,34.147287
EX_acetone_e,-13.086420,20.000000
EX_ac_e,0.000000,0.000000
EX_but_e,0.000000,0.000000
EX_btoh_e,-1.982759,23.095238


In [132]:
# add butyrate to the medium
nj4.reactions.EX_but_e.lower_bound = -10

In [133]:
mv.check_blocked_reactions(nj4, production_reactions)
mv.check_production(nj4, production_reactions)

,minimum,maximum
EX_btoh_e,0.000000,0.000000
EX_etoh_e,19.875820,19.962270
EX_ac_e,0.000000,0.000000
EX_but_e,0.000000,0.000000
EX_acetone_e,2.344472,2.373297


### other stuff - testing. ect:

In [10]:
# useful tool for seeing the intercondections between metabolites!

from utils import model_validation as modval

metabolites = ["xylu__D_c", "xu5p__D_c"]
G = modval.create_graph(nj4, metabolites)
modval.visualise_graph(G, 'nj4_xylose')

nj4_xylose.html


In [11]:
nj4.reactions.TKT1

Reaction identifier,TKT1
Name,Transketolase
Memory address,0x1238d8490
Stoichiometry,r5p_c + xu5p__D_c <=> g3p_c + s7p_c Alpha-D-Ribose 5-phosphate + D-Xylulose 5-phosphate <=> Glyceraldehyde 3-phosphate + Sedoheptulose 7-phosphate
GPR,WP_010964657_1
Lower bound,-1000.0
Upper bound,1000.0


In [4]:
# checking if production of a metabolite is possible at SS

from utils import model_validation as modval

modval.production_possible(nj4, "EX_etoh_e")

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


True

In [3]:
from cobra.io import save_json_model
save_json_model(nj4, "test.json")